In [59]:
import gc
import pandas as pd
import chardet
import csv

pd.options.display.float_format = '{:.10f}'.format

In [2]:
base_path = '/app/data/Desarrollos Norman/'

In [4]:
def detect_encoding(csv_path, n_bytes=100_000):
    with open(csv_path, "rb") as f:
        raw = f.read(n_bytes)
    result = chardet.detect(raw)
    return result

In [5]:
info = detect_encoding(base_path + 'SECOP_II_-_Contratos_Electrónicos_20260204.csv')
print(info)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [3]:
zonificacion_pedagogico = pd.read_excel(
    base_path + '20260212_Plantilla_Matriz_de_Contratacion.xlsx', 
    sheet_name='ZONIFICACIÓN- PEDAGOGICO',
    header=2
)

resumen_oper_directa = pd.read_excel(
    base_path + '20260212_Plantilla_Matriz_de_Contratacion.xlsx', 
    sheet_name = 'RESUMEN- OPER DIRECTA'
)

consolida_ranking = pd.read_excel(
    '/app/data/cruces SECOP/Consolida_Ranking.xlsx',
    sheet_name='Matriz_Ranking'
)

# metaverso = pd.read_excel(
#     '/app/data/Metaverso 2026.xlsx',
#     sheet_name='ZonificacionPais'
# )

# secopii_bk = pd.read_excel(
#     base_path + 'Contratos_ICBF_SECOP_II_20260204V2.xlsx'
# )

In [4]:
zonificacion_pedagogico_plus_resumen_oper_directa = pd.concat([zonificacion_pedagogico, resumen_oper_directa])

cols = [col for col in zonificacion_pedagogico_plus_resumen_oper_directa.columns if col in zonificacion_pedagogico.columns]
zonificacion_pedagogico_plus_resumen_oper_directa = zonificacion_pedagogico_plus_resumen_oper_directa[cols]

zonificacion_pedagogico_plus_resumen_oper_directa['Total_Zona'] = zonificacion_pedagogico_plus_resumen_oper_directa.groupby('ZONA 2026.')['Costo total '].transform('sum')
zonificacion_pedagogico_plus_resumen_oper_directa.loc[zonificacion_pedagogico_plus_resumen_oper_directa.duplicated('ZONA 2026.'), 'Total_Zona'] = 0

print('zonificacion_pedagogico:', zonificacion_pedagogico.shape)
print('resumen_oper_directa:', resumen_oper_directa.shape)
print('zonificacion_pedagogico_plus_resumen_oper_directa:', zonificacion_pedagogico_plus_resumen_oper_directa.shape)

zonificacion_pedagogico: (25992, 32)
resumen_oper_directa: (1672, 28)
zonificacion_pedagogico_plus_resumen_oper_directa: (27664, 32)


In [5]:
consolida_ranking = consolida_ranking[['NUMERO DE CONTRATO', 'NIT OPERADOR', 'CONSECUTIVO MATRIZ CONTRATACION']]
consolida_ranking.drop_duplicates(inplace=True)

In [6]:
concat_plus_ranking = pd.merge(
    left=zonificacion_pedagogico_plus_resumen_oper_directa,
    right=consolida_ranking,
    how='outer',
    left_on=['ZONA 2026.', 'NIT CONTRATISTA 2026'],
    right_on=['CONSECUTIVO MATRIZ CONTRATACION', 'NIT OPERADOR'],
    indicator=True
)

In [7]:
concat_plus_ranking['_merge'].value_counts()

_merge
left_only     20989
both           6733
right_only     1491
Name: count, dtype: int64

In [8]:
concat_plus_ranking.rename(columns={'_merge': 'resultado_cruce_concat_ranking'}, inplace=True)

concat_plus_ranking['resultado_cruce_concat_ranking'] = concat_plus_ranking['resultado_cruce_concat_ranking'].astype(str)

concat_plus_ranking.loc[concat_plus_ranking['resultado_cruce_concat_ranking'] == 'left_only', 'resultado_cruce_concat_ranking'] = 'Fallo: Presente solo en concat'
concat_plus_ranking.loc[concat_plus_ranking['resultado_cruce_concat_ranking'] == 'both', 'resultado_cruce_concat_ranking'] = 'Exitoso'
concat_plus_ranking.loc[concat_plus_ranking['resultado_cruce_concat_ranking'] == 'right_only', 'resultado_cruce_concat_ranking'] = 'Fallo: Presente solo en ranking'

concat_plus_ranking['resultado_cruce_concat_ranking'].value_counts()

resultado_cruce_concat_ranking
Fallo: Presente solo en concat     20989
Exitoso                             6733
Fallo: Presente solo en ranking     1491
Name: count, dtype: int64

In [9]:
print('shape', concat_plus_ranking.shape)
print('columns', concat_plus_ranking.columns)

shape (29213, 36)
columns Index(['ZONA 2026.', 'Regional UDS', 'Municipio UDS', 'Centro Zonal UDS',
       'Codigo Unidad Servicio UDS', 'Unidad Servicio UDS', 'SERVICIO 2026',
       'SERVICIO SIM', 'Componente para la UDS', 'Cupos', 'Total',
       'CONCEPTO DEFINITIVO', 'CONTRATISTA 2026', 'NIT CONTRATISTA 2026',
       'OBSERVACION', 'Conteo_Zona', 'Total_Zona', 'Meses', 'Dias',
       'Dias_Descuento', 'Alistamiento', 'Canasta Mes', 'Variables',
       'Alimentación', 'Dotación', 'Costo total ', 'Apalancamiento_2025',
       'Forma de Contratación', 'NIT CONTRATISTA 20262', 'CONTRATISTA 20263',
       'CTRL FORMA', 'JUNIO', 'NUMERO DE CONTRATO', 'NIT OPERADOR',
       'CONSECUTIVO MATRIZ CONTRATACION', 'resultado_cruce_concat_ranking'],
      dtype='object')


In [10]:
# Se usa porque el secop no está completo en DA hoy 16-02-26
secopii_bk = pd.read_excel(
    '/app/data/Contratos_ICBF_SECOP_II_20260217.xlsx'
)

In [24]:
concat_plus_ranking['NUMERO DE CONTRATO'] = concat_plus_ranking['NUMERO DE CONTRATO'].astype(str)
secopii_bk['Referencia del Contrato'] = secopii_bk['Referencia del Contrato'].astype(str)

In [25]:
cruce_secop = pd.merge(
    left=concat_plus_ranking,
    right=secopii_bk,
    how='left',
    left_on=['NUMERO DE CONTRATO'],
    right_on=['Referencia del Contrato'],
    indicator=True
)

In [26]:
cruce_secop['_merge'].value_counts()

_merge
left_only     23568
both           6013
right_only        0
Name: count, dtype: int64

In [34]:
cruce_secop.rename(columns={'_merge': 'resultado_cruce_secop'}, inplace=True)

cruce_secop['resultado_cruce_secop'] = cruce_secop['resultado_cruce_secop'].astype(str)

cruce_secop.loc[cruce_secop['resultado_cruce_secop'] == 'left_only', 'resultado_cruce_secop'] = 'Fallo: Presente solo en data original'
cruce_secop.loc[cruce_secop['resultado_cruce_secop'] == 'both', 'resultado_cruce_secop'] = 'Exitoso'

cruce_secop['resultado_cruce_secop'].value_counts()

resultado_cruce_secop
Fallo: Presente solo en data original    23568
Exitoso                                   6013
Name: count, dtype: int64

In [90]:
cruce_secop[cruce_secop['Regional UDS'] == 'Antioquia']['resultado_cruce_secop'].value_counts()

resultado_cruce_secop
Fallo: Presente solo en data original    2283
Exitoso                                    58
Name: count, dtype: int64

In [95]:
cruce_secop_exitoso = cruce_secop[cruce_secop['resultado_cruce_secop'] == 'Exitoso']
cruce_secop_fallo = cruce_secop[cruce_secop['resultado_cruce_secop'] != 'Exitoso']

print('cruce_secop_exitoso', cruce_secop_exitoso.shape)
print('cruce_secop_fallo', cruce_secop_fallo.shape)

cruce_secop_exitoso (6013, 124)
cruce_secop_fallo (23568, 124)


In [127]:
cruce_secop_fallo = cruce_secop_fallo.drop(columns=[col for col in secopii_bk.columns])
print('cruce_secop_fallo', cruce_secop_fallo.shape)

cruce_secop_fallo (23568, 37)


In [118]:
# Datos auxiliares suministrados el 19-02-2026 para tratar de maximizar cantidad de registros cruzados con secop
icbfcue_auxiliar = pd.read_excel(
    '/app/data/Johan19022026/ICBFCUEContxRegxVigxDir (23).xlsx',
    sheet_name='ICBFCUEContxRegxVigxDir'
)

consolidado20251702 = pd.read_excel(
    '/app/data/Johan19022026/CONSOLIDADO_2025_17_02_2026.xlsx',
    sheet_name='ReporteSitco'
)

In [119]:
consolidado20251702 = consolidado20251702[
    [
        'Id Contrato', 'Número PACCO', 'Codigo Regional', 'Id Contrato', 'Numero Identificación', 
        'Numero Identificación.1', 'Número Contrato', 'Número PACCO', 'Número de Identificación'
    ]
]

icbfcue_auxiliar = icbfcue_auxiliar[
    [
        'Codigo Regional', 'Documento Representante Legal', 'ID Numero PACCO', 'ID SECOP', 'ID SIIF', 'Id Contrato', 
        'Id Contrato SITCO', 'Identificación EC',  'Identificacion Contacto  Entidad Contratista', 'Valor Total del Contrato'
    ]
]

print('consolidado20251702', consolidado20251702.shape)
print('icbfcue_auxiliar', icbfcue_auxiliar.shape)

consolidado20251702 (24661, 9)
icbfcue_auxiliar (6480, 10)


In [134]:
cruce_secop_fallo['NIT CONTRATISTA 2026'] = cruce_secop_fallo['NIT CONTRATISTA 2026'].astype(str)
consolidado20251702['Número de Identificación'] = consolidado20251702['Número de Identificación'].astype(str)

In [135]:
cruce_secop_fallo_plus_consolidado = pd.merge(
    left=cruce_secop_fallo,
    right=consolidado20251702,
    how='left',
    left_on='NIT CONTRATISTA 2026',
    right_on='Número de Identificación',
    indicator=True
)

cruce_secop_fallo_plus_consolidado['_merge'].value_counts()

_merge
both          146368
left_only       2625
right_only         0
Name: count, dtype: int64

In [128]:
cruce_secop_fallo.columns

Index(['ZONA 2026.', 'Regional UDS', 'Municipio UDS', 'Centro Zonal UDS',
       'Codigo Unidad Servicio UDS', 'Unidad Servicio UDS', 'SERVICIO 2026',
       'SERVICIO SIM', 'Componente para la UDS', 'Cupos', 'Total',
       'CONCEPTO DEFINITIVO', 'CONTRATISTA 2026', 'NIT CONTRATISTA 2026',
       'OBSERVACION', 'Conteo_Zona', 'Total_Zona', 'Meses', 'Dias',
       'Dias_Descuento', 'Alistamiento', 'Canasta Mes', 'Variables',
       'Alimentación', 'Dotación', 'Costo total ', 'Apalancamiento_2025',
       'Forma de Contratación', 'NIT CONTRATISTA 20262', 'CONTRATISTA 20263',
       'CTRL FORMA', 'JUNIO', 'NUMERO DE CONTRATO', 'NIT OPERADOR',
       'CONSECUTIVO MATRIZ CONTRATACION', 'resultado_cruce_concat_ranking',
       'resultado_cruce_secop'],
      dtype='object')

In [131]:
cruce_secop_fallo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23568 entries, 0 to 29575
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ZONA 2026.                       23313 non-null  object 
 1   Regional UDS                     23313 non-null  object 
 2   Municipio UDS                    23313 non-null  object 
 3   Centro Zonal UDS                 23313 non-null  object 
 4   Codigo Unidad Servicio UDS       23313 non-null  object 
 5   Unidad Servicio UDS              23313 non-null  object 
 6   SERVICIO 2026                    23313 non-null  object 
 7   SERVICIO SIM                     23313 non-null  object 
 8   Componente para la UDS           23313 non-null  object 
 9   Cupos                            23313 non-null  float64
 10  Total                            23313 non-null  float64
 11  CONCEPTO DEFINITIVO              23313 non-null  object 
 12  CONTRATISTA 2026       

In [130]:
consolidado20251702.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24661 entries, 0 to 24660
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id Contrato               24661 non-null  int64  
 1   Número PACCO              24652 non-null  float64
 2   Codigo Regional           24661 non-null  int64  
 3   Id Contrato               24661 non-null  int64  
 4   Numero Identificación     454 non-null    float64
 5   Numero Identificación.1   15 non-null     float64
 6   Número Contrato           24661 non-null  int64  
 7   Número PACCO              24652 non-null  float64
 8   Número de Identificación  24648 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 1.7 MB


In [125]:
uniques = {}
for col in sorted(consolidado20251702.columns):
    uniques[col] = consolidado20251702[col].nunique()

print(uniques)

{'Codigo Regional': 34, 'Id Contrato': Id Contrato    24661
Id Contrato    24661
dtype: int64, 'Numero Identificación': 448, 'Numero Identificación.1': 15, 'Número Contrato': 24660, 'Número PACCO': Número PACCO    23893
Número PACCO    23893
dtype: int64, 'Número de Identificación': 14968}


In [126]:
uniques = {}
for col in sorted(icbfcue_auxiliar.columns):
    uniques[col] = icbfcue_auxiliar[col].nunique()

print(uniques)

{'Codigo Regional': 30, 'Documento Representante Legal': 1140, 'ID Numero PACCO': 3471, 'ID SECOP': 1439, 'ID SIIF': 3, 'Id Contrato': 1446, 'Id Contrato SITCO': 1446, 'Identificacion Contacto  Entidad Contratista': 912, 'Identificación EC': 1143, 'Valor Total del Contrato': 1369}


In [ ]:
# sorted(icbfcue_auxiliar.columns)


,Codigo Regional,Documento Representante Legal,ID Numero PACCO,ID SECOP,ID SIIF,Id Contrato,Id Contrato SITCO,Identificación EC,Identificacion Contacto Entidad Contratista,Valor Total del Contrato
0,44,40927128,8693,https://community.secop.gov.co/Public/Tenderin...,9,87208,219313,901196276,40927128.0000000000,2810229341
1,44,40927128,24426,https://community.secop.gov.co/Public/Tenderin...,9,87208,219313,901196276,40927128.0000000000,2810229341
2,44,40927128,32539,https://community.secop.gov.co/Public/Tenderin...,9,87208,219313,901196276,40927128.0000000000,2810229341
3,44,40927128,32539,https://community.secop.gov.co/Public/Tenderin...,9,87208,219313,901196276,40927128.0000000000,2810229341
4,44,40927128,8693,https://community.secop.gov.co/Public/Tenderin...,9,87208,219313,901196276,40927128.0000000000,2810229341


cruce_secop_fallo (23568, 37)


In [36]:
cruce_secop.to_excel('/app/data/cruce_secop.xlsx', index=False)

In [35]:
def get_order_identifiers_as_list(order_identificators_list: list, chunk_size: int = 1000):
    def clean(elem):
        return elem.replace("\n", "").replace("\r", "").strip()

    chunks = [
        order_identificators_list[i:i + chunk_size]
        for i in range(0, len(order_identificators_list), chunk_size)
    ]

    return [
        "(" + ", ".join([f'"{clean(str(elem))}"' for elem in chunk]) + ")"
        for chunk in chunks
    ]

In [45]:
from typing import List, Tuple
import numpy as np

def money_to_float(s: pd.Series) -> pd.Series:
    # 1) a string
    s = s.astype(str)

    # 2) quitar saltos de línea y espacios raros
    s = s.str.replace(r"[\n\r\t]", "", regex=True).str.strip()

    # 3) quitar símbolo $, espacios y separadores de miles
    #    (si el decimal fuera coma, lo tratamos abajo)
    s = (s
         .str.replace("$", "", regex=False)
         .str.replace(" ", "", regex=False)
         .str.replace(",", "", regex=False)
    )

    # 4) convertir a numérico (lo que no se pueda -> NaN)
    return pd.to_numeric(s, errors="coerce")

def fuzzy_merge(merge_column_pairs: List[List[str]], pct: float, matriz: pd.DataFrame, secop: pd.DataFrame) -> pd.DataFrame:
    # merge_column_pairs: [valor_l, valor_r, texto_l, texto_r]]
    merge_result = []
    for pair in merge_column_pairs:
        valor_l = pair[0]; valor_r = pair[1]
        texto_l = pair[2]; texto_r = pair[3]
        
        # --- 0) ids para controlar 1-1 ---
        L = matriz.copy()
        R = secop.copy()

        L[valor_l] = money_to_float(L[valor_l])
        R[valor_r] = money_to_float(R[valor_r])

        L[valor_l] = L[valor_l].astype(float)
        R[valor_r] = R[valor_r].astype(float)

        L["_id_l"] = np.arange(len(L))
        R["_id_r"] = np.arange(len(R))

        # --- 1) números ---
        L[valor_l] = pd.to_numeric(L[valor_l], errors="coerce")
        R[valor_r] = pd.to_numeric(R[valor_r], errors="coerce")

        L = L.dropna(subset=[valor_l]).sort_values(valor_l).copy()
        R = R.dropna(subset=[valor_r]).sort_values(valor_r).copy()

        # --- 2) candidato por cercanía ---
        cand = pd.merge_asof(
            L,
            R,
            left_on=valor_l,
            right_on=valor_r,
            direction="nearest"
        )

        # --- 3) texto: normaliza a dígitos (NIT/documento) ---
        def only_digits(x):
            if pd.isna(x):
                return ""
            return "".join(ch for ch in str(x) if ch.isdigit())

        l_doc = cand[texto_l].map(only_digits)
        r_doc = cand[texto_r].map(only_digits)

        txt_ok = (l_doc == r_doc) & (r_doc != "")

        # --- 4) tolerancia porcentual ---
        den = cand[valor_l].abs()
        diff = (cand[valor_l] - cand[valor_r]).abs()
        tol_ok = np.where(den.eq(0), diff.eq(0), diff <= (pct * den))

        cand = cand[txt_ok & tol_ok].copy()

        # --- 5) score: menor diferencia porcentual = mejor ---
        cand["diff_abs"] = diff[txt_ok & tol_ok]
        cand["diff_pct"] = np.where(den[txt_ok & tol_ok].eq(0), 0, cand["diff_abs"] / den[txt_ok & tol_ok])

        # --- 6) resolver colisiones para que sea 1-1 ---
        # Paso A: cada L se queda con su mejor R
        cand = cand.sort_values(["_id_l", "diff_pct"]).drop_duplicates("_id_l", keep="first")

        # Paso B: si varios L apuntan al mismo R, nos quedamos con el mejor (y descartamos el resto)
        cand = cand.sort_values(["_id_r", "diff_pct"]).drop_duplicates("_id_r", keep="first")

        # (Opcional) Paso C: volver a asegurar unicidad por L (por si el paso B dejó huecos)
        cand = cand.sort_values(["_id_l", "diff_pct"]).drop_duplicates("_id_l", keep="first")

        out = cand.drop(columns=["diff_abs", "diff_pct", "_id_r", "_id_l"])

        merge_result.append(out)
    
    return pd.concat(merge_result)

In [82]:
pct = 0.8  # 3% tolerancia

merge_column_pairs = [
    ['Total_Zona', 'Valor del Contrato', 'NIT OPERADOR', 'Documento Proveedor'],   
    ['Total_Zona', 'Valor del Contrato', 'NIT OPERADOR', 'Identificación Representante Legal'],
]

cruce_secop_fallo['Total_Zona'] = cruce_secop_fallo['Total_Zona'].astype(str)
cruce_secop_fallo['NIT OPERADOR'] = cruce_secop_fallo['NIT OPERADOR'].astype(str)
secopii_bk['Documento Proveedor'] = secopii_bk['Documento Proveedor'].astype(str)
secopii_bk['Identificación Representante Legal'] = secopii_bk['Identificación Representante Legal'].astype(str)
secopii_bk['Valor del Contrato'] = secopii_bk['Valor del Contrato'].astype(str)

merge = fuzzy_merge(merge_column_pairs, pct, cruce_secop_fallo, secopii_bk)

In [83]:
merge.shape

(0, 124)

In [ ]:
cruce_secop_fallo[cruce_secop_fallo['Total_Zona'] > 0]

(0, 124)

In [85]:
1074043863 / 1052984178.9299628735

1.0200000004667096

In [64]:
cruce_secop_fallo[(cruce_secop_fallo['Total_Zona'] > 0) & ~(cruce_secop_fallo['NIT OPERADOR'].isna())][['Total_Zona', 'NIT OPERADOR']].head(15)

,Total_Zona,NIT OPERADOR
0,1052984178.9299628735,900937582
4,2385275070.7718677521,900937582
1603,219027156.0614084303,901292472
1990,561031189.5227161646,901797179
3867,106496889.8121470958,901665692
4212,201735536.7242124975,820001914
4216,284005028.7063036561,826000799
9044,240268798.2701634765,901436211
9288,201235007.8492751122,900256500
9525,462258752.8714328408,901378700


In [72]:
cruce_secop_fallo[cruce_secop_fallo['NIT OPERADOR'] == '9004330802'][['Total_Zona', 'NIT OPERADOR']]

,Total_Zona,NIT OPERADOR
9991,985249220.0019243956,9004330802
9992,0.0000000000,9004330802
9993,0.0000000000,9004330802
9994,0.0000000000,9004330802
9995,0.0000000000,9004330802
9996,0.0000000000,9004330802
9997,0.0000000000,9004330802
9998,0.0000000000,9004330802
9999,0.0000000000,9004330802
10000,0.0000000000,9004330802


In [84]:
secopii_bk[(secopii_bk['Identificación Representante Legal'].str.contains('900937582')) | (secopii_bk['Documento Proveedor'].str.contains('900937582'))]['Valor del Contrato']

1441      $678,054,303
3310    $1,074,043,863
Name: Valor del Contrato, dtype: object

In [43]:
sorted(cruce_secop_fallo.columns)

['Alimentación',
 'Alistamiento',
 'Apalancamiento_2025',
 'CONCEPTO DEFINITIVO',
 'CONSECUTIVO MATRIZ CONTRATACION',
 'CONTRATISTA 2026',
 'CONTRATISTA 20263',
 'CTRL FORMA',
 'Canasta Mes',
 'Centro Zonal UDS',
 'Codigo Unidad Servicio UDS',
 'Componente para la UDS',
 'Conteo_Zona',
 'Costo total ',
 'Cupos',
 'Dias',
 'Dias_Descuento',
 'Dotación',
 'Forma de Contratación',
 'JUNIO',
 'Meses',
 'Municipio UDS',
 'NIT CONTRATISTA 2026',
 'NIT CONTRATISTA 20262',
 'NIT OPERADOR',
 'NUMERO DE CONTRATO',
 'OBSERVACION',
 'Regional UDS',
 'SERVICIO 2026',
 'SERVICIO SIM',
 'Total',
 'Total_Zona',
 'Unidad Servicio UDS',
 'Variables',
 'ZONA 2026.',
 'resultado_cruce_concat_ranking',
 'resultado_cruce_secop']

In [30]:
# metaverso = pd.read_excel(
#     base_path + 'Metaverso 2026.xlsx',
#     sheet_name=None
# )
# metaverso.keys()

# del metaverso
# gc.collect()